# CSV File Analysis and SQL Schema
The purpose of this notebook is to check the quality of the data that's downloaded and see if any cleaning/transformations are needed. I will also start exploring what the SQL schema will look like.

In [1]:
import json

from pathlib import Path 
import pandas as pd

# this notebook assumes you're running Jupyter from root
ROOT = Path().resolve().parent
DATA_DIR = ROOT / "data"
MOVIES_CSV = DATA_DIR / "movies.csv"
MOVIE_DETAILS_CSV = DATA_DIR / "movie_details.csv"
CAST_CSV = DATA_DIR / "cast.csv"
GENRES_CSV = DATA_DIR / "genres.csv"

## CSV Exploration
### Movies CSV
I explored this file a little in the first EDA notebook, but I will perform a more thorough one here and consider what features to use for the database.

In [2]:
# need python engine
df_movies = pd.read_csv(MOVIES_CSV, engine="python")
print(df_movies.head())

   adult                     backdrop_path                genre_ids     id  \
0  False  /jhk6D8pim3yaByu1801kMoxXFaX.jpg             [28, 18, 12]     98   
1  False   /zvmsyAMr3cVDdIu7UvDLSmRXlF.jpg          [35, 18, 10749]  22705   
2  False  /7isarjYDEKZ5t1CgcvbuqEUby8P.jpg                     [27]   9532   
3  False  /mZj8EUr6F1x2PWZjKPxaeYd5WRw.jpg  [12, 16, 35, 10751, 14]  11688   
4  False  /uHZRTGMFb1RLmgWcqlIOZsGbDCT.jpg                     [35]   4247   

  original_language            original_title  \
0                en                 Gladiator   
1                it             Tra(sgre)dire   
2                en         Final Destination   
3                en  The Emperor's New Groove   
4                en               Scary Movie   

                                            overview  popularity  \
0  After the death of Emperor Marcus Aurelius, hi...     14.7103   
1  While scouting out apartments in London for he...     16.3672   
2  After a teenager has a terrif

In [3]:
print(df_movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58473 entries, 0 to 58472
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              58473 non-null  bool   
 1   backdrop_path      54391 non-null  object 
 2   genre_ids          58473 non-null  object 
 3   id                 58473 non-null  int64  
 4   original_language  58473 non-null  object 
 5   original_title     58472 non-null  object 
 6   overview           57946 non-null  object 
 7   popularity         58473 non-null  float64
 8   poster_path        58287 non-null  object 
 9   release_date       58473 non-null  object 
 10  title              58472 non-null  object 
 11  video              58473 non-null  bool   
 12  vote_average       58473 non-null  float64
 13  vote_count         58473 non-null  int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 5.5+ MB
None


In [4]:
print(df_movies.describe())

                 id    popularity  vote_average    vote_count
count  5.847300e+04  58473.000000  58473.000000  58473.000000
mean   3.853717e+05      1.117523      6.047465    317.876165
std    3.398420e+05     12.856676      1.072113   1387.823057
min    8.000000e+00      0.000000      1.200000     10.000000
25%    7.157500e+04      0.207700      5.400000     16.000000
50%    3.308890e+05      0.389900      6.141000     32.000000
75%    5.841230e+05      0.808000      6.800000    103.000000
max    1.515343e+06   2649.567200     10.000000  37742.000000


In [5]:
# let's check for missing values
print("Count of missing values:")
print(df_movies.isnull().sum())
print("\nPercent missing values:")
print(df_movies.isnull().sum()/len(df_movies))

Count of missing values:
adult                   0
backdrop_path        4082
genre_ids               0
id                      0
original_language       0
original_title          1
overview              527
popularity              0
poster_path           186
release_date            0
title                   1
video                   0
vote_average            0
vote_count              0
dtype: int64

Percent missing values:
adult                0.000000
backdrop_path        0.069810
genre_ids            0.000000
id                   0.000000
original_language    0.000000
original_title       0.000017
overview             0.009013
popularity           0.000000
poster_path          0.003181
release_date         0.000000
title                0.000017
video                0.000000
vote_average         0.000000
vote_count           0.000000
dtype: float64


In [6]:
# overall, it's a pretty complete dataset with most fields filled
# the attribute with the most missing values is backdrop, but it's unneeded for now

# let's check entries with a missing title
missing_title = (df_movies["original_title"].isnull()) | (df_movies["title"].isnull())
print(df_movies[missing_title])

       adult backdrop_path genre_ids       id original_language  \
49799  False           NaN  [28, 18]  1161605                en   

      original_title                                           overview  \
49799            NaN  A hitman is tasked to take out ex-mobsters whe...   

       popularity poster_path release_date title  video  vote_average  \
49799       0.242         NaN   2021-05-21   NaN  False           9.2   

       vote_count  
49799          44  


In [7]:
# there's only one and it seems to be an obscure title, so it shouldn't affect
# our future analysis much

# lastly let's check duplicates
print(df_movies[df_movies.duplicated()])

Empty DataFrame
Columns: [adult, backdrop_path, genre_ids, id, original_language, original_title, overview, popularity, poster_path, release_date, title, video, vote_average, vote_count]
Index: []


### Movie Details CSV
Now let's check the movie details csv

In [8]:
df_movie_details = pd.read_csv(MOVIE_DETAILS_CSV, engine="python")

In [9]:
print(df_movie_details.head())

   adult                     backdrop_path  \
0  False  /Ar7QuJ7sJEiC0oP3I8fKBKIQD9u.jpg   
1  False  /7isarjYDEKZ5t1CgcvbuqEUby8P.jpg   
2  False  /uHZRTGMFb1RLmgWcqlIOZsGbDCT.jpg   
3  False   /zvmsyAMr3cVDdIu7UvDLSmRXlF.jpg   
4  False  /mZj8EUr6F1x2PWZjKPxaeYd5WRw.jpg   

                               belongs_to_collection     budget  \
0  {"id": 1069584, "name": "Gladiator Collection"...  103000000   
1  {"id": 8864, "name": "Final Destination Collec...   23000000   
2  {"id": 4246, "name": "Scary Movie Collection",...   19000000   
3                                                NaN    2100000   
4  {"id": 178117, "name": "The Emperor's New Groo...  100000000   

                                              genres homepage     id  \
0  [{"id": 28, "name": "Action"}, {"id": 18, "nam...      NaN     98   
1                     [{"id": 27, "name": "Horror"}]      NaN   9532   
2                     [{"id": 35, "name": "Comedy"}]      NaN   4247   
3  [{"id": 35, "name": "Comedy"}

In [10]:
df_movie_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57708 entries, 0 to 57707
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  57708 non-null  bool   
 1   backdrop_path          53615 non-null  object 
 2   belongs_to_collection  7132 non-null   object 
 3   budget                 57708 non-null  int64  
 4   genres                 57708 non-null  object 
 5   homepage               18286 non-null  object 
 6   id                     57708 non-null  int64  
 7   imdb_id                57253 non-null  object 
 8   origin_country         57708 non-null  object 
 9   original_language      57708 non-null  object 
 10  original_title         57707 non-null  object 
 11  overview               57172 non-null  object 
 12  popularity             57708 non-null  float64
 13  poster_path            57523 non-null  object 
 14  production_companies   57708 non-null  object 
 15  pr

In [11]:
df_movie_details.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,5.770800e+04,5.770800e+04,57708.000000,5.770800e+04,57708.000000,57708.000000,57708.000000
mean,3.980601e+06,3.760472e+05,1.019226,1.042993e+07,94.099899,6.042712,319.576298
std,1.830418e+07,3.297459e+05,2.416987,6.808384e+07,31.099745,1.073133,1392.866088
min,0.000000e+00,8.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,10.000000
25%,0.000000e+00,7.038050e+04,0.250400,0.000000e+00,85.000000,5.400000,16.000000
50%,0.000000e+00,3.245400e+05,0.464200,0.000000e+00,94.000000,6.134500,32.000000
75%,0.000000e+00,5.741512e+05,0.949400,0.000000e+00,107.000000,6.800000,104.000000
max,4.654000e+08,1.471337e+06,155.658200,2.923706e+09,999.000000,10.000000,37670.000000


In [12]:
# the max vote_count value is a little high, let's see if it's plausible
# (i.e. a big blockbuster)
df_movie_details[df_movie_details["vote_count"] == 37670]

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
15806,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,NaN,160000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",https://www.warnerbros.com/movies/inception,27205,tt1375666,"[""US"", ""GB""]",en,...,2010-07-15,839030630,148,"[{""english_name"": ""English"", ""iso_639_1"": ""en""...",Released,Your mind is the scene of the crime.,Inception,False,8.369,37670


In [13]:
# it's Inception, so very believable
# just like the discover/movie dataset, this seems to be pretty complete with
# most missing values occuring in "non-essential" attributes such as homepage,
# tagline, etc

# there are several nested jsons in this file, namely "belongs_to_collection" and "production_companies"
# let's get a sample of how that looks
json.loads(df_movie_details["belongs_to_collection"][0])

{'id': 1069584,
 'name': 'Gladiator Collection',
 'poster_path': '/bk6nx2rGNdlKtBsB9XcrclVKItv.jpg',
 'backdrop_path': '/1VdLvSIeHuwqCT13H9EafxCacGB.jpg'}

In [14]:
json.loads(df_movie_details["production_companies"][0])

[{'id': 33,
  'logo_path': '/3wwjVpkZtnog6lSKzWDjvw2Yi00.png',
  'name': 'Universal Pictures',
  'origin_country': 'US'},
 {'id': 1645,
  'logo_path': '/6Ry6uNBaa0IbbSs1XYIgX5DkA9r.png',
  'name': 'Scott Free Productions',
  'origin_country': 'GB'},
 {'id': 14440,
  'logo_path': '/gk9zllyMFnaVBWTdZpm7VcfQrHC.png',
  'name': 'Red Wagon Entertainment',
  'origin_country': 'US'},
 {'id': 21904, 'logo_path': None, 'name': 'Mill Film', 'origin_country': 'GB'},
 {'id': 7,
  'logo_path': '/vru2SssLX3FPhnKZGtYw00pVIS9.png',
  'name': 'DreamWorks Pictures',
  'origin_country': 'US'}]

In [15]:
# both fields look interesting, but for now we can keep the production_companies in
# our initial database schema, as it has the two fields probably most relevant for our
# initial analysis. the collection field might require me to rework my movie script
# and ingest from the collections endpoint

# last check: duplicates
df_movie_details[df_movie_details.duplicated()]

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [16]:
### Cast CSV

In [17]:
df_cast = pd.read_csv(CAST_CSV, engine="python")

In [18]:
df_cast.head()

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
0,False,2,934,Acting,Russell Crowe,Russell Crowe,4.0960,/rsxGCRtPu42uKDJZlz7qknvz8h6.jpg,8,Maximus,52fe4217c3a36847f8003435,0,98
1,False,2,73421,Acting,Joaquin Phoenix,Joaquin Phoenix,3.8118,/u38k3hQBDwNX0VA22aQceDp9Iyv.jpg,9,Commodus,52fe4217c3a36847f8003439,1,98
2,False,1,935,Acting,Connie Nielsen,Connie Nielsen,3.3614,/lvQypTfeH2Gn2PTbzq6XkT2PLmn.jpg,10,Lucilla,52fe4217c3a36847f800343d,2,98
3,False,2,936,Acting,Oliver Reed,Oliver Reed,1.3842,/dWfotc1X71wNCGyPO9hXpv8U9Gw.jpg,11,Proximo,52fe4217c3a36847f8003441,3,98
4,False,2,194,Acting,Richard Harris,Richard Harris,3.1177,/lCvcVMuxrg1f5A8OMqY9AqkkcZR.jpg,12,Marcus Aurelius,52fe4217c3a36847f8003445,4,98


In [19]:
df_cast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108750 entries, 0 to 1108749
Data columns (total 13 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   adult                 1108750 non-null  bool   
 1   gender                1108750 non-null  int64  
 2   id                    1108750 non-null  int64  
 3   known_for_department  1108749 non-null  object 
 4   name                  1108750 non-null  object 
 5   original_name         1108750 non-null  object 
 6   popularity            1108750 non-null  float64
 7   profile_path          766796 non-null   object 
 8   cast_id               1108750 non-null  int64  
 9   character             1050509 non-null  object 
 10  credit_id             1108750 non-null  object 
 11  order                 1108750 non-null  int64  
 12  movie_id              1108750 non-null  int64  
dtypes: bool(1), float64(1), int64(5), object(6)
memory usage: 102.6+ MB


In [20]:
df_cast.describe()

,gender,id,popularity,cast_id,order,movie_id
count,1.108750e+06,1.108750e+06,1.108750e+06,1.108750e+06,1.108750e+06,1.108750e+06
mean,1.208934e+00,1.397959e+06,4.716132e-01,6.685634e+01,1.723717e+01,3.511497e+05
std,8.307885e-01,1.376516e+06,1.493642e+00,1.741735e+02,2.164802e+01,3.274482e+05
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+00
25%,0.000000e+00,1.005670e+05,3.870000e-02,9.000000e+00,5.000000e+00,5.227400e+04
50%,1.000000e+00,1.228155e+06,1.565000e-01,2.200000e+01,1.100000e+01,2.893350e+05
75%,2.000000e+00,2.086114e+06,4.569000e-01,4.800000e+01,2.200000e+01,5.495090e+05
max,3.000000e+00,5.569390e+06,2.663570e+02,1.473000e+03,4.500000e+02,1.471337e+06


In [21]:
# let's check the known_for_department feature
print(df_cast["known_for_department"].unique())

['Acting' 'Production' 'Lighting' 'Directing' 'Crew' 'Art' 'Writing'
 'Camera' 'Sound' 'Costume & Make-Up' 'Creator' 'Editing' 'Visual Effects'
 nan]


In [22]:
# interestingly, i was expecting most of the folk here to be labeled as 'acting'
# let's get an idea of how many in 'cast' are labeled otherwise
df_cast_nonactors = df_cast[df_cast["known_for_department"] != "Acting"]
print(len(df_cast_nonactors)/len(df_cast))

0.04525456595264938


In [23]:
# let's see what the first 5 entries look like
df_cast_nonactors.head()

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
26,False,2,2219097,Production,Billy Dowd,Billy Dowd,0.0694,NaN,270,Narrator,63723864798e0600dc36ff44,26,98
36,False,0,2156204,Lighting,Neil Roche,Neil Roche,0.0322,NaN,310,Roman Soldier (uncredited),637251898fdda900cd07cc0c,36,98
37,False,0,2151711,Directing,Paul Sacks,Paul Sacks,0.0409,NaN,311,"Catapult Commander Shouts ""Loose"" (uncredited)",63725198798e06007fe6b26d,37,98
39,False,2,1434599,Crew,Brian Smyj,Brian Smyj,0.1944,/zpVyJRhBQQZ3bc2COTOtv5iUmv6.jpg,315,Coliseum Gladiator (uncredited),63725240028420007a1d031a,39,98
63,False,2,21069,Production,Randy Stone,Randy Stone,0.2476,/n2pLehPKyJaYBN0aayajnBariAj.jpg,94,Flight Attendant,59cbf5dec3a368774e0136c8,19,9532


In [24]:
# looks like non-actors seem to be casted as extras. because of this,
# we can safely not include the known_for_department feature for now

# duplicates check
df_cast[df_cast.duplicated()]

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id


In [25]:
# let's see if some actors had multiple roles
df_cast[df_cast.duplicated(["id", "movie_id"], keep=False)].sort_values(by="popularity", ascending=False)

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
179697,True,1,146550,Acting,Sora Aoi,蒼井そら,36.115,/xo9JfzQhIdGBaa70O36lWfAT0ud.jpg,32,Girl Being Discovered by Talent Scout,5a1178c7c3a368628304f7ef,28,31512
179708,True,1,146550,Acting,Sora Aoi,蒼井そら,36.115,/xo9JfzQhIdGBaa70O36lWfAT0ud.jpg,60,NaN,6646f0bef6a9c96dae3d9a0e,39,31512
429292,True,1,232271,Acting,Jynx Maze,Jynx Maze,16.339,/1uoPh4O60hBbpEc3AkU9luhXFm2.jpg,21,Self,65d7461ced2ac2017c369736,15,115189
429287,True,1,232271,Acting,Jynx Maze,Jynx Maze,16.339,/1uoPh4O60hBbpEc3AkU9luhXFm2.jpg,14,Herself,5a1d5c320e0a264cbc067f86,10,115189
201529,True,2,141115,Acting,Max Parodi,Max Parodi,8.028,/riuKtMiQGwTLPDQdaLuyi8D4ABu.jpg,22,Max,5a2759940e0a264ccd0fc398,26,103631
...,...,...,...,...,...,...,...,...,...,...,...,...,...
937782,False,0,4363831,Acting,Fara Abidah,Fara Abidah,0.000,NaN,76,Kulin's Fans,654b3b0c41a561336b79bfa3,54,572667
488533,False,0,4897991,Art,Mark Young,Mark Young,0.000,NaN,37,Shadow Creature,66c64dcc9e34d1fe7ffa3b8d,15,484626
488530,False,0,4897991,Art,Mark Young,Mark Young,0.000,NaN,34,Friendly Driver,66c64da0b8dd9602fdd1eaf1,12,484626
1094258,False,0,1305579,Directing,Pablo Riera,Pablo Riera,0.000,NaN,17,Jander,5eb49ad30cb3350020c9a929,11,490039


### Genres CSV
Lastly, let's check out the genres csv. It's a fairly small file so we can just print it

In [26]:
df_genres = pd.read_csv(GENRES_CSV, engine="python")
print(df_genres)

       id             name
0      28           Action
1      12        Adventure
2      16        Animation
3      35           Comedy
4      80            Crime
5      99      Documentary
6      18            Drama
7   10751           Family
8      14          Fantasy
9      36          History
10     27           Horror
11  10402            Music
12   9648          Mystery
13  10749          Romance
14    878  Science Fiction
15  10770         TV Movie
16     53         Thriller
17  10752              War
18     37          Western


### CSV Summary
Overall, it looks like the CSV files are largely complete with few missing values in key features. There are some such as backdrop_path and profile_path that might be useful in the future, but we can exclude those for now. The following are the features I'll keep for now.

- **Movies CSV**:
  
genre_ids, id, original_language, original_title, overview, popularity, release_date, title, vote_average, vote_count
- **Movie Details CSV**:

budget, genres, id, origin_country, original_language, original_title, production_companies, revenue, runtime, tagline, status
- **Cast CSV**:

id, name, original_name, popularity, character, order, movie_id

- **Genres CSV**

id, name